In [1]:
with open('Main_Train.csv', 'w') as creating_new_csv_file: 
   pass 
print("Empty File Created Successfully")

Empty File Created Successfully


In [2]:
import tensorflow as tf
import os
import glob
import numpy as np
import cv2
import random

In [3]:
train_files = []

for x in os.walk('/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train'):
    for y in glob.glob(os.path.join(x[0], '*.png')):
        train_files.append(y)
random.shuffle(train_files)
print(len(train_files))

9152


In [4]:
valid_files = []
for x in os.walk('/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/validation'):
    for y in glob.glob(os.path.join(x[0], '*.png')):
        valid_files.append(y)
random.shuffle(valid_files)
print(len(valid_files))

1000


In [5]:
def get_input(path):
    im = cv2.imread(path)
    im = cv2.resize(im, (244, 244))
    #im = im.reshape(-1,244,244,3)
    
    return(im)
 
def get_output(path):
    labaF = [1,0]
    labaT = [0,1]
    label = (path.split('/')[-1]).split('_')[0]
    if 'fake' == label:
        return labaF
    else:
        return labaT

In [6]:
#mod = Xception(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling='Avg', classes=1000)

mod = tf.keras.applications.DenseNet121(include_top=True, weights='imagenet', input_shape=(224,224,3), pooling='Avg', classes=1000)
for layer in mod.layers:
    mod.trainable = True
layer = tf.keras.layers.Dense(512, activation='relu')(mod.layers[-2].output)
layer = tf.keras.layers.Dense(2, activation='softmax')(layer)
mod = tf.keras.models.Model(inputs=mod.input, outputs=layer)

33188688/33188688 [==============================] - 0s 0us/step


In [7]:
def image_generator(files, batch_size, resize=None):
 
      while True:
          batch_paths  = np.random.choice(a = files, size = batch_size)
          batch_x = []
          batch_y = [] 
          
          for input_path in batch_paths:
              input = get_input(input_path)
              output = get_output(input_path)
              if resize is not None:
                input = cv2.resize(input, resize)
              batch_x.append(input)
              batch_y.append(output)
          batch_x = np.array(batch_x)
          batch_y = np.array(batch_y)
          yield batch_x, batch_y

In [8]:
#ATTENTION APPROACH 2(USING AMNET MODEL WITH INCEPTION AS CORE CNN) WITH 512 ATTENTION NODE (ON FACESWAP)
mod.compile('Adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy', tf.keras.metrics.AUC()])
mod.summary()
checkpoint_filepath = '/kaggle/working/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')
 
history = mod.fit(image_generator(train_files, batch_size = 32), epochs = 100, steps_per_epoch = 40, validation_data = image_generator(valid_files, batch_size = 32), validation_steps = 15, callbacks=[model_checkpoint_callback])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [10]:
#####TEST SCRIPT#####

import os
import cv2
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix


class test():
    def get_filpath(self, dirname):
        test_files = []
        for x in os.walk(dirname):
            for y in glob.glob(os.path.join(x[0], '*.png')):
                test_files.append(y)

        return test_files
        
    def get_input(self, path):
        input = cv2.imread(path)
        input = cv2.resize(input, (224, 224))
        input = input.reshape(-1, 224, 224, 3)
        return input
    
    def get_output(self, path):
        labaF = [1,0]
        labaT = [0,1]
        label = (path.split('/')[7]).split('_')[0]
        #print(label)
        if 'fake' == label:
            return labaF
        else:
            return labaT

    def load_model(self, modpath):
        mod = tf.keras.models.load_model(modpath)
        return mod

    def test(self, paths, modpath):
        true_label = []
        predicted = []
        mod = self.load_model(modpath)
        positive = 0
        for path in paths:
            im = self.get_input(path)
            lab = self.get_output(path)
            
            pred_label = np.argmax(mod.predict(im))
            predicted.append(pred_label)

            real_label = np.argmax(lab)
            true_label.append(real_label)
            if pred_label == real_label:
                positive = positive + 1
            #print("Filepath : {}, Predicted Label = {}, True Label = {}".format(path, pred_label, real_label))

        FPR, TPR, threshold1 = roc_curve(true_label, predicted)
        print(FPR, TPR, threshold1)
        aucSci = roc_auc_score(true_label, predicted)
        print('Auc score scikit: ', aucSci)
        pd.DataFrame(confusion_matrix(true_label, predicted), columns=['Predicted Fake', "Predicted Real"], index=['Actual Fake', 'Actual Real'])
        print(pd)
        return positive/len(paths), FPR, TPR


    def run(self, dirname, modpath):
        paths = self.get_filpath(dirname)
        random.shuffle(paths)
        res, FPR, TPR = self.test(paths, modpath)

        return res, FPR, TPR

In [11]:
if __name__=='__main__':
    dirname = '/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/testImage'  #PATH OF TEST DATA DIRECTORY
    modpath = '/kaggle/working/checkpoint'  #PATH OF TRAINED MODEL
    acc, FPR, TPR = test().run(dirname, modpath)
    print("Test Accuracy = {}".format(acc))

1/1 [==============================] - 0s 34ms/step
[0.         0.03823529 1.        ] [0.         0.85393258 1.        ] [2 1 0]
Auc score scikit:  0.9078486450760079
<module 'pandas' from '/opt/conda/lib/python3.7/site-packages/pandas/__init__.py'>
Test Accuracy = 0.9247104247104247


In [22]:
# Making listing of files & store it in .csv file
import glob
import pandas as pd
CSVFile = '/kaggle/input/deepfake/test.csv'
lstFiles = glob.glob1("/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/testImage", "*.png")
print(len(lstFiles))
df = pd.DataFrame(lstFiles)
df.to_csv(CSVFile, header = False, index=False)

518


OSError: [Errno 30] Read-only file system: '/kaggle/input/deepfake/test.csv'

In [ ]:
#Feature Extraction using Xception

In [13]:
import pandas as pd

imgList = pd.read_csv('/kaggle/input/labels/train.csv', index_col=False, header = None)
imgList = imgList[imgList.columns[0]].tolist()

FeatureFileName = '/kaggle/working/Main_Train.csv'
print(len(imgList))

9152


In [14]:
def get_input(path):
    im = cv2.imread(path)
    im = cv2.resize(im,(224,224))
    im = im.reshape(-1,224,224,3)
    
    return(im)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(mod,show_shapes=True,to_file="abc.png")

In [15]:
import tensorflow as tf
mod = tf.keras.models.load_model('/kaggle/working/checkpoint')
mod = tf.keras.Model(inputs=mod.input, outputs=mod.layers[-3].output)

In [16]:
mod.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                            

In [10]:
def convertFeatureDatatoCSV(all_features):
    tmpList = []
    #Last column store class value
    
    feature_dimention = len(all_features[0]) - 1
    print(feature_dimention)
    #Prepare column header string for csv file
    #for heading in range (feature_dimention):
    #    strng = 'attr-' + str(heading)
    #    tmpList.append(strng)
    #tmpList.append('class')
    
    #all_features.insert(0, tmpList)
    df = pd.DataFrame(all_features)
    df.to_csv(FeatureFileName, header = False, index=False)

In [ ]:
import cv2
feature = []
i=0
for img in imgList:
    if 'fake' in img:
        typ = 1
    else:
        typ = 0
    imgPath = '/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/' + img
    print(imgPath)
    out = mod.predict(get_input(imgPath))
    out = out.tolist()
    flatten_list = [j for sub in out for j in sub]
    flatten_list.append(typ)
    feature.append(flatten_list)
    print(i)
    i += 1
convertFeatureDatatoCSV(feature)

/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0000_frame117.png
1/1 [==============================] - 2s 2s/step
0
/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0000_frame234.png
1/1 [==============================] - 0s 34ms/step
1
/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0001_frame150.png
1/1 [==============================] - 0s 35ms/step
2
/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0001_frame75.png
1/1 [==============================] - 0s 34ms/step
3
/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0002_frame174.png
1/1 [==============================] - 0s 33ms/step
4
/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0002_frame87.png
1/1 [==============================] - 0s 33ms/step
5
/kaggle/input/deepfake/CelebImageDataset/Celeb Image Dataset/train/fake_id0_id16_0005_

In [ ]:
#Train & Test using Xception features [Designed mlp]

In [ ]:
import tensorflow as tf
from keras.layers import Input

mod = tf.keras.Input(shape = (2048,))
layer = tf.keras.layers.Dense(512, activation='relu')(mod)
layer = tf.keras.layers.Dense(2, activation='sigmoid')(layer)
model = tf.keras.models.Model(inputs=mod, outputs=layer)

In [ ]:
import pandas as pd
import random

train_data = pd.read_csv('/content/drive/MyDrive/KhalidTemp/CelebImageDataset/xception_celeb_train_feature.csv', index_col=False, header = None)
train_data = train_data.sample(frac = 1)
y_train = train_data[train_data.columns[-1]]
x_train = train_data.drop(train_data.columns[[-1]], axis=1)

test_data = pd.read_csv('/content/drive/MyDrive/KhalidTemp/CelebImageDataset/xception_celeb_test_feature.csv', index_col=False, header = None)
test_data = test_data.sample(frac = 1)
y_test = test_data[test_data.columns[-1]]
x_test = test_data.drop(test_data.columns[[-1]], axis=1)


In [ ]:
Y_train = []
Y_test = []
for val in y_train:
    if val == 1:
        Y_train.append([1,0])
    else:
        Y_train.append([0,1])
        
for val in y_test:
    if val == 1:
        Y_test.append([1,0])
    else:
        Y_test.append([0,1])
        
y_train = np.asarray(Y_train)
y_test = np.asarray(Y_test)

In [ ]:
# Configure the model and start training
model.compile('Adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy', tf.keras.metrics.AUC()])

history = model.fit(x_train, y_train, epochs=200, batch_size=32, steps_per_epoch = 40, verbose=1, validation_steps = 15, validation_batch_size=32, validation_split=0.2)

In [ ]:
#Training MLP

model.compile('Adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy', tf.keras.metrics.AUC()])
 
checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')
 
#history = model.fit((train_files, batch_size = 32), epochs = 50, steps_per_epoch = 40, callbacks=[model_checkpoint_callback])
history = model.fit(x_train, y_train, epochs=30, batch_size=32, verbose=1, validation_split=0.2)

In [ ]:
# Test the model after training
test_results = model.evaluate(x_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

17/17 [==============================] - 0s 5ms/step - loss: 0.1874 - accuracy: 0.9556 - auc_8: 0.9906

Test results - Loss: 0.18736720085144043 - Accuracy: 0.9555984735488892%
